# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataset
print(data.head())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view


In [2]:
#Your code here
# Check for any missing values
print(data.isnull().sum())

# Check the unique values in the 'group' column
print(data['group'].unique())

# Check the unique values in the 'action' column
print(data['action'].unique())

# Count the number of viewers and clickers
viewers = data[data['action'] == 'view'].shape[0]
clickers = data[data['action'] == 'click'].shape[0]
print("Number of viewers:", viewers)
print("Number of clickers:", clickers)

# Check for any anomalies
anomalies = data[(data['action'] == 'click') & (~data['id'].isin(data[data['action'] == 'view']['id']))]
print("Anomalies (click without view):")
print(anomalies)

# Check for overlap between control and experiment groups
overlap = data.groupby('id')['group'].nunique().reset_index()
overlap_count = overlap[overlap['group'] > 1].shape[0]
print("Number of users in both groups:", overlap_count)

timestamp    0
id           0
group        0
action       0
dtype: int64
['experiment' 'control']
['view' 'click']
Number of viewers: 6328
Number of clickers: 1860
Anomalies (click without view):
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []
Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [3]:
#Your code here
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(data['group'], data['action'])

#contingency table
print(contingency_table)

# chi-square test for independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

# test statistics and p-value
print("Chi-square statistic:", chi2)
print("P-value:", p)

action      click  view
group                  
control       932  3332
experiment    928  2996
Chi-square statistic: 3.636160051233291
P-value: 0.056537191086915774


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
#Your code here
control_click_through_rate = contingency_table.loc['control', 'click'] / contingency_table.loc['control'].sum()
experiment_expected_clicks = control_click_through_rate * contingency_table.loc['experiment'].sum()
experiment_expected_clicks

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [5]:
#Your code here
import numpy as np
experiment_actual_clicks = contingency_table.loc['experiment', 'click']
experiment_group_size = contingency_table.loc['experiment'].sum()

z_score = (experiment_actual_clicks - experiment_expected_clicks) / np.sqrt(experiment_group_size * control_click_through_rate * (1 - control_click_through_rate))

z_score

2.71607302278784

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [6]:
#Your code here
#Your code here
from scipy.stats import norm
p_value = 1 - norm.cdf(z_score)
p_value

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

The analysis confirms significant difference in click-through rates between control and experimental groups, favoring the experimental homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.